In [ ]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [ ]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- OR summary ---
    if "OR" in d.columns:
        or_mean = d["OR"].mean()
        or_std = d["OR"].std(ddof=1)
        df_out.loc[feature, "OR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minOR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxOR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["OR"].mean()
    or_std  = d["OR"].std(ddof=1)
    df_out.loc[idx, "OR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minOR"] = d["2.5%"].min()
    df_out.loc[idx, "maxOR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [ ]:
data = pd.read_csv('data/BBAG-cross.csv')

In [ ]:
data = data[data.country != 'Slovakia'].reset_index(drop =  True)

In [ ]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


## Odd ratios adding co-vars

In [ ]:
covar_list = (
                'gender_equal_l', 'Polution_conc_inv', 'Eq', 'free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est', 'GDP_per_capita',
                'number_leng_inst', 'number_stable_inst', 'distance', 'Migration', 'Educ_doct', 'Educ_bach'
)


In [ ]:
def fit_logit_or(df, features, ycol='GAP_bin'):
    c_df = df[features + [ycol]].dropna()
    X = c_df[features]
    y = c_df[ycol]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    

    scaler = MinMaxScaler((0.05, 0.95))
    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    X_train_scaled['intercept'] = 1
    X_test_scaled['intercept'] = 1

    model = sm.Logit(y_train, X_train_scaled).fit(disp=0)
    OR = np.exp(model.params[features[0]])
    return OR

results_merge_df_all = data.copy()
results_all = []

for i in vars_:

    df_base = results_merge_df_all[[i, 'GAP_bin'] + list(covar_list)]

    OR_base = fit_logit_or(df_base, [i])
    logOR_ref = np.log(OR_base)

    selected = []
    remaining = list(covar_list)

    step = 0

    while len(remaining) > 0:

        deltas = []

        for c in remaining:
            features = [i] + selected + [c]
            OR_new = fit_logit_or(df_base, features)
            delta = abs(np.log(OR_new) - logOR_ref)

            deltas.append({
                'Feature': i,
                'Step': step + 1,
                'Candidate_covar': c,
                'OR_ref': np.exp(logOR_ref),
                'OR_new': OR_new,
                'Delta_logOR': delta
            })

        df_deltas = pd.DataFrame(deltas)
        best = df_deltas.sort_values('Delta_logOR').iloc[0]

        # actualizar
        selected.append(best['Candidate_covar'])
        remaining.remove(best['Candidate_covar'])
        logOR_ref = np.log(best['OR_new'])
        step += 1

        best['Selected_covar'] = best['Candidate_covar']
        best['Selected_so_far'] = ', '.join(selected)
        results_all.append(best)


In [ ]:
df_stepwise_OR = pd.DataFrame(results_all)
df_stepwise_OR